# Data preparation

This Jupyter notebook aims to prepare datasets from different sources, each with its own composition and granularity. We will unify the localization granularity of all datasets using UHF42 (there are 42 in New York), representing the most detailed geographic reference available in our data. Subsequently, we will merge all datasets based on the UHF42 code.

Initially, we will focus on preparing the dataset containing air pollutant concentrations and their related health outcomes. To discern disparities between UHF42 neighborhoods, we will add data on various external factors that are more likely to contribute to outdoor pollutants.

In [1]:
import pandas as pd
from IPython.display import clear_output
import geopandas as gpd
from shapely.wkt import loads
import os

In [2]:
def map_zipcode_to_uhf(zipcode : int):
    """
    Map a zipcode into UHF42 code
    :param zipcode: the zipcode to translate into a UHF42 code
    """
    uhf_mapping = {
        101: [10463, 10471],
        102: [10464, 10466, 10469, 10470, 10475],
        103: [10458, 10467, 10468],
        104: [10461, 10462, 10465, 10472, 10473],
        105: [10453, 10457, 10460],
        106: [10451, 10452, 10456],
        107: [10454, 10455, 10459, 10474],
        201: [11211, 11222],
        202: [11201, 11205, 11215, 11217, 11231],
        203: [11212, 11213, 11216, 11233, 11238],
        204: [11207, 11208],
        205: [11220, 11232],
        206: [11204, 11218, 11219, 11230],
        207: [11203, 11210, 11225, 11226],
        208: [11234, 11236, 11239],
        209: [11209, 11214, 11228],
        210: [11223, 11224, 11229, 11235],
        211: [11206, 11221, 11237],
        301: [10031, 10032, 10033, 10034, 10040],
        302: [10026, 10027, 10030, 10037, 10039],
        303: [10029, 10035],
        304: [10023, 10024, 10025],
        305: [10021, 10028, 10044, 10128],
        306: [10001, 10011, 10018, 10019, 10020, 10036],
        307: [10010, 10016, 10017, 10022],
        308: [10012, 10013, 10014],
        309: [10002, 10003, 10009],
        310: [10004, 10005, 10006, 10007, 10038, 10280],
        401: [11101, 11102, 11103, 11104, 11105, 11106],
        402: [11368, 11369, 11370, 11372, 11373, 11377, 11378],
        403: [11354, 11355, 11356, 11357, 11358, 11359, 11360],
        404: [11361, 11362, 11363, 11364],
        405: [11374, 11375, 11379, 11385],
        406: [11365, 11366, 11367],
        407: [11414, 11415, 11416, 11417, 11418, 11419, 11420, 11421],
        408: [11412, 11423, 11430, 11432, 11433, 11434, 11435, 11436, 11001],
        409: [11004, 11005, 11411, 11413, 11422, 11426, 11427, 11428, 11429],
        410: [11691, 11692, 11693, 11694, 11695, 11697],
        501: [10302, 10303, 10310],
        502: [10301, 10304, 10305],
        503: [10314],
        504: [10306, 10307, 10308, 10309, 10312],
    }
   
    for uhf, zipcodes in uhf_mapping.items(): 
        if zipcode in zipcodes:
            return uhf
    return None

## 1. Air quality and related Health outcomes data

The primary dataset to prepare is sourced from [NYC Government Open Data](https://data.cityofnewyork.us/Environment/Air-Quality/c3uy-2p5r).
It contains various variables, including mean concentrations of fine particulates (PM2.5, O3 and NO2) and health burdens associated with each particulate matter. 
The census time frame spanned from 2007 to 2020. 
The granularity of locations varies : boroughs, Community Districts, UH34, and more precisely, UHF42.

We will keep a granularity of UHF42.

In [3]:
path_file = os.path.join('data', 'Air_Quality_20240210.csv')
air_quality_df = pd.read_csv(path_file)
air_quality_df

,Unique ID,Indicator ID,Name,Measure,Measure Info,Geo Type Name,Geo Join ID,Geo Place Name,Time Period,Start_Date,Data Value,Message
0,172653,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2011,12/01/2010,25.30,NaN
1,172585,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,203,Bedford Stuyvesant - Crown Heights,Annual Average 2009,12/01/2008,26.93,NaN
2,336637,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,204,East New York,Annual Average 2015,01/01/2015,19.09,NaN
3,336622,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,103,Fordham - Bronx Pk,Annual Average 2015,01/01/2015,19.76,NaN
4,172582,375,Nitrogen dioxide (NO2),Mean,ppb,UHF34,104,Pelham - Throgs Neck,Annual Average 2009,12/01/2008,22.83,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16213,130750,647,Outdoor Air Toxics - Formaldehyde,Annual average concentration,µg/m3,UHF42,211,Williamsburg - Bushwick,2005,01/01/2005,3.10,NaN
16214,130780,647,Outdoor Air Toxics - Formaldehyde,Annual average concentration,µg/m3,Borough,5,Staten Island,2005,01/01/2005,2.30,NaN
16215,131020,652,Cardiac and respiratory deaths due to Ozone,Estimated annual rate,"per 100,000",UHF42,504,South Beach - Tottenville,2005-2007,01/01/2005,7.50,NaN
16216,131026,652,Cardiac and respiratory deaths due to Ozone,Estimated annual rate,"per 100,000",Borough,5,Staten Island,2005-2007,01/01/2005,7.80,NaN


In [4]:
# Create a new DataFrame containing 'UHF42' and the corresponding 'Borough'
# It will be used to construct a final dataset from the original dataset 'air_quality_df'
uhf42_df = pd.DataFrame()
uhf42_df["UHF42"] = air_quality_df.loc[(air_quality_df["Geo Type Name"] == "UHF42")]["Geo Join ID"].unique()
uhf42_df["Borough"] = "Staten Island"
uhf42_df["Borough"].loc[uhf42_df["UHF42"]<=410] = "Queens"
uhf42_df["Borough"].loc[uhf42_df["UHF42"]<=310] = "Manhattan"
uhf42_df["Borough"].loc[uhf42_df["UHF42"]<=211] = "Brooklyn"
uhf42_df["Borough"].loc[uhf42_df["UHF42"]<=117] = "Bronx"
clear_output()

In [5]:
def add_feature(original_dataframe: pd.DataFrame,
                uhf42_df: pd.DataFrame, 
                feature_name: str, 
                new_feature_name:str
                )->pd.DataFrame:
    """
    This function takes the name of a variable and returns a dataframe containing the pivot table of that variable. The data is given by UHF42.
    
    :param original_dataframe: The dataframe containing the original Air Quality data.
    :param uhf42_df: The dataframe containing UHF42 and corresponding Borough name.
    :param feature_name: The name of the variable for which to create a pivot table.
    :param new_feature_name: The new name to use for the variable.
    """

    # Gets a small dataframe for the given feature_name by UHF42
    df_slice= original_dataframe.loc[(original_dataframe["Geo Type Name"] == "UHF42") & (original_dataframe["Name"] == feature_name)]
    
    # If the column Measure has one unique value (mean, Number per km2...)
    if df_slice.Measure.nunique() == 1:
        df_slice= df_slice[["Geo Join ID", "Data Value", "Time Period"]]
        
        # Create a pivot table containing :
        # Column for number of UHF42
        # Columns for all available time periods
        # The data values are the value of the given feature_name 
        df_slice= pd.pivot_table(df_slice, values="Data Value", index="Geo Join ID", columns="Time Period").reset_index()
        
        # Renaming the columns as follows : new_feature_name | time_period
        rename_dict= {current_feature_name:new_feature_name for current_feature_name, new_feature_name in zip(df_slice.columns[1:], [f"{new_feature_name} | {current_feature_name}" for current_feature_name in df_slice.columns[1:]])}
        df_slice= df_slice.rename(columns=rename_dict)  
        
        # Merging the pivot table with the dataframe containing UHF42 & borough
        final_df= pd.merge(uhf42_df, df_slice, how="left", left_on="UHF42", right_on="Geo Join ID")
        final_df.drop(columns="Geo Join ID", inplace=True)
        return final_df
    
    # If the column Measure has more than one unique value (mean, Number per km2...)
    else:
        # Retrieving the first measure
        df_slice_1= df_slice.loc[df_slice["Measure"] == df_slice.Measure.unique()[0]]
        df_slice_1= df_slice_1[["Geo Join ID", "Data Value", "Time Period"]]
        df_slice_1= pd.pivot_table(df_slice_1, values="Data Value", index="Geo Join ID", columns="Time Period").reset_index()
        rename_dict= {current_feature_name:new_feature_name for current_feature_name, new_feature_name in zip(df_slice_1.columns[1:], [f"{new_feature_name} | {df_slice.Measure.unique()[0]} | {current_feature_name}" for current_feature_name in df_slice_1.columns[1:]])}
        df_slice_1= df_slice_1.rename(columns=rename_dict)

        # Retrieving the second measure
        df_slice_2= df_slice.loc[df_slice["Measure"] == df_slice.Measure.unique()[1]]
        df_slice_2= df_slice_2[["Geo Join ID", "Data Value", "Time Period"]]
        df_slice_2= pd.pivot_table(df_slice_2, values="Data Value", index="Geo Join ID", columns="Time Period").reset_index()
        rename_dict= {current_feature_name:new_feature_name for current_feature_name, new_feature_name in zip(df_slice_2.columns[1:], [f"{new_feature_name} | {df_slice.Measure.unique()[1]} | {current_feature_name}" for current_feature_name in df_slice_2.columns[1:]])}
        df_slice_2= df_slice_2.rename(columns=rename_dict)

        #merging the 1st and 2nd measure with the uhf42 dataframe
        final_df= pd.merge(uhf42_df, df_slice_1, how="left", left_on="UHF42", right_on="Geo Join ID")
        final_df.drop(columns="Geo Join ID", inplace=True)
        final_df= pd.merge(final_df, df_slice_2, how="left", left_on="UHF42", right_on="Geo Join ID")
        final_df.drop(columns="Geo Join ID", inplace=True)  
    return final_df

In [6]:
# OAPs Concentration

oap_df = add_feature(air_quality_df, uhf42_df, 'Nitrogen dioxide (NO2)', 'NO2')
oap_df = add_feature(air_quality_df, oap_df, 'Fine particles (PM 2.5)', 'PM2.5')
oap_df = add_feature(air_quality_df, oap_df, 'Ozone (O3)', 'O3')

In [7]:
# PM2.5 health Outcomes

pm2_5_health = add_feature(air_quality_df, oap_df, 'Asthma emergency department visits due to PM2.5', 'PM2.5_AEDV')
pm2_5_health = add_feature(air_quality_df, pm2_5_health, 'Cardiovascular hospitalizations due to PM2.5 (age 40+)', 'PM2.5_CH')
pm2_5_health = add_feature(air_quality_df, pm2_5_health, 'Deaths due to PM2.5', 'PM2.5_D')
pm2_5_health = add_feature(air_quality_df, pm2_5_health, 'Respiratory hospitalizations due to PM2.5 (age 20+)', 'PM2.5_RH')

In [8]:
# O3 health Outcomes

o3_health = add_feature(air_quality_df, pm2_5_health, 'Asthma hospitalizations due to Ozone', 'O3_AH')
o3_health = add_feature(air_quality_df, o3_health, 'Cardiac and respiratory deaths due to Ozone', 'O3_CRD')
air_quality_df_final = add_feature(air_quality_df, o3_health, 'Asthma emergency departments visits due to Ozone', 'O3_AEDV')

In [9]:
air_quality_df_final

,UHF42,Borough,NO2 | Annual Average 2009,NO2 | Annual Average 2010,NO2 | Annual Average 2011,NO2 | Annual Average 2012,NO2 | Annual Average 2013,NO2 | Annual Average 2014,NO2 | Annual Average 2015,NO2 | Annual Average 2016,...,O3_CRD | 2012-2014,O3_CRD | 2015-2017,O3_AEDV | Estimated annual rate (age 18+) | 2005-2007,O3_AEDV | Estimated annual rate (age 18+) | 2009-2011,O3_AEDV | Estimated annual rate (age 18+) | 2012-2014,O3_AEDV | Estimated annual rate (age 18+) | 2015-2017,O3_AEDV | Estimated annual rate (under age 18) | 2005-2007,O3_AEDV | Estimated annual rate (under age 18) | 2009-2011,O3_AEDV | Estimated annual rate (under age 18) | 2012-2014,O3_AEDV | Estimated annual rate (under age 18) | 2015-2017
0,206,Brooklyn,24.79,23.43,23.32,20.87,20.23,20.41,19.74,18.87,...,5.0,4.8,11.6,15.4,15.1,14.2,12.4,22.7,24.0,22.5
1,106,Bronx,29.35,26.38,26.67,24.23,23.73,22.56,22.28,21.08,...,3.7,3.9,124.0,113.1,134.5,124.9,171.2,202.0,248.9,220.8
2,209,Brooklyn,22.54,21.48,21.62,19.27,18.58,19.10,18.62,17.78,...,6.5,6.3,12.9,13.7,14.5,12.8,16.9,24.9,24.2,25.9
3,210,Brooklyn,19.90,19.01,18.98,17.21,16.36,17.63,17.08,15.62,...,8.0,8.5,22.6,24.0,26.8,27.7,33.4,42.3,42.4,46.2
4,410,Queens,14.68,14.07,14.27,13.12,12.67,13.77,13.55,11.72,...,9.4,7.7,58.8,59.7,72.6,63.0,135.2,137.4,91.0,84.8
5,107,Bronx,26.39,24.10,24.58,22.27,21.54,21.14,20.91,19.91,...,3.7,3.6,130.1,119.1,146.8,130.1,169.4,206.3,237.6,225.0
6,207,Brooklyn,25.14,23.64,23.56,21.16,20.57,20.47,19.72,18.86,...,4.7,4.8,49.8,65.9,67.6,62.8,93.5,129.2,137.4,119.3
7,208,Brooklyn,20.22,19.10,19.21,17.22,16.65,17.00,16.52,15.30,...,5.9,6.0,38.3,49.8,52.2,48.5,79.4,97.7,98.8,88.9
8,407,Queens,22.93,21.70,22.21,20.01,19.65,19.19,18.68,17.80,...,4.1,3.8,27.0,35.0,33.9,28.3,63.5,89.0,83.7,61.2
9,201,Brooklyn,26.95,25.08,25.84,23.63,22.85,22.43,21.69,21.44,...,3.5,3.1,27.0,26.9,32.1,27.6,42.7,58.6,50.5,36.5


## 2. External factors related to outdoor pollutants

To understand the variability of outdoor pollutants between different neighborhoods, we will incorporate additional datasets concerning potential influential factors contributing to or causing one neighborhood to exhibit higher pollution levels, deaths, hospitalizations, and department visits than another.

This data is included : 
- Traffic density
- Poverty levels
- Distribution of parks
- Number of restaurants
- Building energy consumption

Same as air quality dataset, we will keep data by UH42 for all the following datasets.

### a. Traffic data

Actors such as traffic density plays a significant role in our study. Therefore, we integrated a dataset containing traffic records from Automated Traffic Recorders (ATR) used in NY to collect traffic sample volume counts at bridge crossings and roadways.
This data was obtained from [NYC Government Open Data](https://data.cityofnewyork.us/Transportation/Automated-Traffic-Volume-Counts/7ym2-wayt/about_data?fbclid=IwAR0libv-PXblJ_jFkDTj4m5nvP8W6eqRqOxvkNmpN2EOP1701mY9S2AzJQ8).
Each line in the dataset corresponds to the traffic volume and the geographical coordinates of the sensor. We will translate the geographical coordinates into UHF42 and calculate the mean traffic volume for each UHF42.

In [10]:
path_file = os.path.join('data', 'traffic_dataset.csv')
traffic_df = pd.read_csv(path_file)

In [11]:
# Create GeoDataFrame and add geometry that will be used after to get the UHF42
traffic_df['geometry'] = traffic_df['WktGeom'].apply(loads)
traffic_df = gpd.GeoDataFrame(traffic_df, geometry='geometry')

# Read UHF42 data that will be used to find the UHF42 corresponding to each geometry in traffic dataset
uhf42_data = gpd.read_file('UHF42/UHF42.shp')
uhf42_data['id'] = uhf42_data['id'].astype(int)

# Function to be applied on the dataset to find the corresponding UHF42 of a given geometry
def find_uhf42(row):
    for _, uhf42_row in uhf42_data.iterrows():
        if row['geometry'].within(uhf42_row['geometry']):
            return uhf42_row['id']
    return None

traffic_df['UHF42'] = traffic_df.apply(find_uhf42, axis=1)

# Drop NaN values of UHF42 (that could not been found) and convert UHF42 column into int
traffic_df.dropna(subset=['UHF42'], inplace=True)
traffic_df['UHF42'] = traffic_df['UHF42'].astype(int)
traffic_df.drop(traffic_df[traffic_df['UHF42'] == 0].index, inplace=True)

# Drop unnecessary columns
traffic_df.drop(['RequestID', 'Boro', 'Yr', 'SegmentID', 'WktGeom', 'street', 'fromSt', 'toSt', 'Direction', 'geometry'], axis=1, inplace=True)

# Group by UHF42 and calculate mean traffic
traffic_df = traffic_df.groupby(["UHF42"])['Vol'].mean().reset_index()
traffic_df.rename(columns={'Vol': 'traffic volume'}, inplace=True)

In [12]:
traffic_df

,UHF42,traffic volume
0,101,104.625229
1,102,104.812212
2,103,96.396747
3,104,102.807893
4,105,91.613596
5,106,75.994612
6,107,86.785480
7,201,77.877853
8,202,80.952125
9,203,75.367312


### b. Poverty data

We made an assumption that socio-economic factors may influence health outcomes, so we included data on poverty also obtained from [NYC Government Open Data](https://a816-dohbesp.nyc.gov/IndicatorPublic/data-explorer/economic-conditions/?id=103&fbclid=IwAR23HtmQWzd6-LhBlWnOq8VxLbNb2Gp1omKIlstQDZYc8G75_wut8eSq6RI#display=summary), covering the years from 2017 to 2021.

In [13]:
path_file = os.path.join('data', 'NYC EH Data Portal - Neighborhood poverty (filtered).csv')
poverty_df = pd.read_csv(path_file)

In [14]:
# keep only UHF42
poverty_df = poverty_df.loc[(poverty_df['GeoTypeDesc'] == 'UHF 42')]

# Drop some unnecessary columns
poverty_df.drop(['TimePeriod','GeoTypeDesc','GeoRank','Geography','Number'], axis=1, inplace=True)
poverty_df.rename(columns={'GeoID': 'UHF42', 'Percent' : 'poverty'}, inplace=True)

In [15]:
poverty_df

,UHF42,poverty
6,101,16.2
7,102,14.8
8,103,27.7
9,104,20.3
10,105,33.6
11,106,35.8
12,107,36.3
13,201,17.1
14,202,13.7
15,203,21.6


### c. Parks data

Additionally, parks have a significant role in improving air quality. So, we added a dataset mapping the distribution of parks across NY from [NYC Government Open Data](https://data.cityofnewyork.us/Recreation/Parks-Properties-Map/krz2-j7bn).

In [16]:
path_file = os.path.join('data', 'Parks_Properties_20240210.csv')
parks_df = pd.read_csv(path_file)

In [17]:
parks_df = parks_df[['ACRES', 'ZIPCODE']]

#ZIPCODE column may contain multiple zip codes if the park is large
parks_df = parks_df[parks_df['ZIPCODE'].str.len() % 5 == 0]

# this function seperates all the zipcodes
def split_zipcode(zipcode):
    return [zipcode[i:i+5] for i in range(0, len(zipcode), 5)]

new_rows = []
for index, row in parks_df.iterrows():
    zipcode = str(row['ZIPCODE'])
    if len(zipcode) > 5 :
        sub_zipcodes = split_zipcode(zipcode)
        acres_per_sub_zip = row['ACRES'] / len(sub_zipcodes)
        for sub_zip in sub_zipcodes:
            new_rows.append({'ZIPCODE': sub_zip, 'ACRES': acres_per_sub_zip})

new_data = pd.DataFrame(new_rows)
parks_df = pd.concat([parks_df, new_data], ignore_index=True)
parks_df = parks_df[parks_df['ZIPCODE'].str.len() <= 5]
parks_df['ZIPCODE'] = parks_df['ZIPCODE'].astype(int)

In [18]:
# Mapping a zipcode into a UHF42
parks_df['UHF42'] = parks_df['ZIPCODE'].apply(map_zipcode_to_uhf)
parks_df.drop(['ZIPCODE'], axis=1, inplace=True)
parks_df.dropna(subset=['UHF42'])

# counting the number of parks in each UHF42
parks_df = parks_df.groupby('UHF42')['ACRES'].sum().reset_index()
parks_df.rename(columns={'ACRES': 'parks superficy'}, inplace=True)

In [19]:
parks_df

,UHF42,parks superficy
0,101.0,510.749500
1,102.0,2073.038095
2,103.0,831.405833
3,104.0,2193.195298
4,105.0,431.531417
5,106.0,232.747286
6,107.0,145.922000
7,201.0,135.207000
8,202.0,773.623667
9,203.0,248.416133


### d. Restaurants data

According to the [National Emissions Inventory](https://www.epa.gov/air-emissions-inventories/national-emissions-inventory-nei), there is a significant rise in emissions from commercial cooking. Thus, we integrated a dataset containing restaurant distribution in NY neighborhoods from [NYC Government Open Data](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j).

In [20]:
path_file = os.path.join('data', 'DOHMH_New_York_City_Restaurant_Inspection_Results_20240210.csv')
restaurants_df = pd.read_csv(path_file)

In [21]:
# Mapping a zipcode into a UHF42
restaurants_df['UHF42'] = restaurants_df['ZIPCODE'].apply(map_zipcode_to_uhf)
restaurants_df.dropna(subset=['UHF42'])
restaurants_df = restaurants_df['UHF42'].value_counts().reset_index()
restaurants_df.columns = ['UHF42', 'number of restaurants']

In [22]:
restaurants_df

,UHF42,number of restaurants
0,306.0,19047
1,309.0,11314
2,308.0,11223
3,402.0,11183
4,202.0,10918
5,307.0,10420
6,401.0,8729
7,403.0,8401
8,203.0,6420
9,207.0,5480


### e. Building consumption data

Moreover, building heating are important in our study. So, we integrated data containing building energy consumption reported to the Environmental Protection Agency in 2016 from  [Kaggle](https://www.kaggle.com/datasets/claytonmiller/new-york-city-buildings-energy-consumption-survey?fbclid=IwAR26gcwjnmJIlwTzyQ9WDNeG0bRFUjtOM2r594mZRD_2WUVkVXXPiryrP_w)

In [23]:
path_file = os.path.join('data', 'nyc_benchmarking_disclosure_data_reported_in_2016.xlsx')
building_df = pd.read_excel(path_file, sheet_name=0)

In [24]:
# extracting only interesting columns
building_df = building_df[['Zip Code', 
                           'Natural Gas Use (kBtu)',
                           'Total GHG Emissions (Metric Tons CO2e)']]

# Creating a copy of the DataFrame
building_df = building_df.copy()

# Mapping a zipcode into a UHF42
building_df['UHF42'] = building_df['Zip Code'].apply(map_zipcode_to_uhf)

# Remove the 'Zip Code' column
building_df.drop(columns=['Zip Code'], inplace=True)

# Group by UHF42 and calculate mean traffic for each UHF42
building_df = building_df.groupby(["UHF42"]).mean().reset_index()

In [25]:
building_df

,UHF42,Natural Gas Use (kBtu),Total GHG Emissions (Metric Tons CO2e)
0,101.0,5.686365e+07,3256.320818
1,102.0,8.300407e+06,882.632222
2,103.0,4.003515e+06,578.734061
3,104.0,1.297178e+07,1244.671250
4,105.0,5.162494e+06,625.539617
5,106.0,6.319824e+06,650.261364
6,107.0,5.462431e+06,497.035000
7,201.0,7.392840e+08,37093.683333
8,202.0,8.219237e+06,870.854452
9,203.0,7.006118e+06,608.898049


## 3. Merging datasets

We will merge all datasets based on UHF42 and sore it.

In [26]:
# Merge building_df with restaurants_df on 'UHF42'
merged_df = pd.merge(building_df, restaurants_df, on='UHF42', how='left')

# Merge merged_df with parks_df on 'UHF42'
merged_df = pd.merge(merged_df, parks_df, on='UHF42', how='left')

# Merge merged_df with poverty_df on 'UHF42'
merged_df = pd.merge(merged_df, poverty_df, on='UHF42', how='left')

# Merge merged_df with traffic_df on 'UHF42'
merged_df = pd.merge(merged_df, traffic_df, on='UHF42', how='left')

# Merge merged_df with air_quality_df on 'UHF42'
merged_df = pd.merge(merged_df, air_quality_df_final, on='UHF42', how='left')

In [27]:
path_file = os.path.join('data', 'full_dataset.csv')
merged_df.to_csv(path_file)